## Looking at CAM Budgets

First we set up our environment as before:


In [1]:
# !!! Make sure you are using the cmpi6-2019.10 kernel !!!!

# Add ldcpy root to system path (MODIFY FOR YOUR LDCPY CODE LOCATION)
import sys

sys.path.insert(0, '/glade/u/home/abaker/repos/ldcpy')
import ldcpy

# Display output of plots directly in Notebook
%matplotlib inline
# Automatically reload module if it is editted
%reload_ext autoreload
%autoreload 2

# silence warnings
import warnings

warnings.filterwarnings("ignore")

In [2]:
# for Cheyenne
from dask_jobqueue import PBSCluster

cluster = PBSCluster(
    queue="regular",
    walltime="02:00:00",
    project="NIOW0001",
    memory="109GB",
    resource_spec="select=1:ncpus=9:mem=109GB",
    cores=36,
    processes=9,
)


# scale as needed
cluster.adapt(minimum_jobs=1, maximum_jobs=30)
cluster

Collections for variables needed for budgets


In [3]:
from dask.distributed import Client

# Connect client to the remote dask workers
client = Client(cluster)
client

Client Scheduler: tcp://10.148.6.55:43163 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abaker/proxy/{port}/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
fsnt_col = ldcpy.open_datasets(
    "cam-fv",
    ["FSNT"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
flnt_col = ldcpy.open_datasets(
    "cam-fv",
    ["FLNT"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
precc_col = ldcpy.open_datasets(
    "cam-fv",
    ["PRECC"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
precl_col = ldcpy.open_datasets(
    "cam-fv",
    ["PRECL"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
fsns_col = ldcpy.open_datasets(
    "cam-fv",
    ["FSNS"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
flns_col = ldcpy.open_datasets(
    "cam-fv",
    ["FLNS"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
shflx_col = ldcpy.open_datasets(
    "cam-fv",
    ["SHFLX"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
lhflx_col = ldcpy.open_datasets(
    "cam-fv",
    ["LHFLX"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)
qflx_col = ldcpy.open_datasets(
    "cam-fv",
    ["QFLX"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/extra_aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
    ],
    ["orig", "con", "mg", "agg", "xagg"],
)

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07

dataset size in GB 0.07



In [7]:
# group collections
all_ds = ldcpy.combine_datasets(
    [fsnt_col, flnt_col, qflx_col, precc_col, precl_col, lhflx_col, shflx_col, flns_col, fsns_col]
)

In [10]:
ldcpy.cam_budgets(all_ds, ["orig", "con", "mg", "agg", "xagg"], significant_digits=6)

,orig,con,mg,agg,xagg
restom,0.574528,0.574511,0.574342,0.574487,0.478375
precip,3.50037e-08,3.50037e-08,3.50038e-08,3.50045e-08,3.49449e-08
e-p,6.0489,6.0489,6.04892,6.04897,6.01777
ressurf,1.42711,1.42714,1.42712,1.42667,0.922054
